In [19]:
sequence_leangh = 4000
shellBlock = 10
shellIteration = 4

## **E91 protocol implemention**

The E91 protocol is a quantum key distribution (QKD) method that leverages the principles of quantum entanglement to establish a secure communication channel between two parties, typically referred to as Alice and Bob. The protocol begins with a central source generating pairs of entangled particles. One particle from each pair is sent to Alice, and the other is sent to Bob. Alice and Bob are the two parties who wish to establish a secure communication channel. Upon receiving the particles, Alice independently chooses random bases from the set  **S = {Xcos(θ) + Zsin(θ) | θ = 0, π/4, π/2}** to measure her particles. Bob will do the same thing, but with one small difference: he will choose from the basis set **S = {Xcos(θ) + Zsin(θ) | θ = π/4, π/2, 3π/4}**.

They will then use a public channel to check their chosen measurement bases. If they measure a pair of entangled qubits with the same basis, they will use the outcome as their weak key. On the other hand, if the qubits were measured in different bases, they would use the results in a Bell inequality to check if the Bell inequality holds or not. If it holds, it means that the qubits were not fully entangled. This could imply that an eavesdropper is intercepting the qubits or that the qubits have been depolarized by the environment.

We have implemented these two entanglement killers in the code. You will see an explanation for different parts of the code in each section.

In [9]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit,transpile, assemble
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import DensityMatrix, partial_trace, Statevector
from qiskit_aer import Aer
import numpy as np
import random

from shell import shell_protocol
from privacyAmlification import generate_hash_family, universal_hash_with_index, secure_hash_using_hmac, get_random_bytes
from depolarization import bipartite_gate, depolarizing_kraus, channel
from BellTest import BellTest

Qiskit can only measure in Z-basis. For measureing in an arbitrary basis, we should first make a Unitary gate that converts basis V to the  basis U. We can calculate this gate by the formuala below:
$$
\sum_i \left| V_i \right\rangle \left\langle U_i \right|
$$

As we do the same for a 2 dimentional space in the ChageBasis function:

\>>>A = [[1, 0], [0, 1]]

\>>>B = [[0, 1], [1, 0]]

\>>>ChangeBasis(A, B)

array([[ 0.70710678,  0.70710678],
       [-0.70710678,  0.70710678]])

In [10]:
def ChangeBasis(first_basis, second_basis):
    first_eigenvalues, first_eigenvectors = np.linalg.eig(first_basis)
    second_eigenvalues, second_eigenvectors = np.linalg.eig(second_basis)
    U = np.outer(second_eigenvectors[0], first_eigenvectors[0]) + np.outer(second_eigenvectors[1], first_eigenvectors[1])
    return U

This function takes a boolian variable eve, which determines the accurence of an evesdropper
it always makes a 3 qubit system, if eve is listening, it would entagle the thirsd qubit to the other 2 qubits, but if eve is not listening it would return a quantum circuit of 3 qubits which the first 2 are entangled but the third one is completly free.

\>>>QC = GenerateBellState(eve = True)

\>>> QC

<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x20c095a6210>

\>>>Statevector(qc)

Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j],
            dims=(2, 2, 2))


In [11]:
def GenerateBellState(eve = False):
    qc = QuantumCircuit(3, 3)
    qc.h(0)
    qc.cx(0, 1)
    if eve:
        qc.cx(1, 2)
        
    return qc

This function will measure a QuantumCircuit and returns a string that contains the resauly of measurment.

\>>>measure(QC)
"111"

In [12]:
def measure(qc):      
    backend = Aer.get_backend('qasm_simulator')
    transpiled_qc = transpile(qc, backend)
    qobj = assemble(transpiled_qc, shots=1)
    result = backend.run(qc, shots=1).result()
    
    counts = result.get_counts()
    measured_bit = counts.most_frequent()
    return measured_bit

This function will return the result of measurment as the previous function, but it takes  $\rho$ instead of QuantumCircuit

\>>> rho = [[0.5 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0],
            [0 0 0 0 0 0 0 0.5]]
            
\>>> measure_depolarizarion(rho)

"000"

In [13]:
def measure_depolarization(rho):
    outcome = ""
    cdf = [0]
    for i in range(0, rho.shape[0]):
        cdf.append(cdf[i] + rho[i][i])

    rand = np.random.rand()
    
    for i in range(len(cdf)):
        if rand < cdf[i]:
            outcome = format(i-1, '03b')
            break

    return outcome
    
    

This function is an inner function tht will be called by ***E91*** function and implements the E91 protocol. In the Ende it will return  Alices and Bobs chosen basis, and their measured bits.

In [14]:
def calculate(sequence_leangh, qc, basis, depolarization=False, p=1/2, q=1/2):
    Alice_basis = random.choices([0, 1, 2], [1/3, 1/3, 1/3], k=sequence_leangh)
    Bob_basis = random.choices([1, 2, 3], [1/3, 1/3, 1/3], k=sequence_leangh)
    measured_bits = []
    for i in range(sequence_leangh):

        if depolarization == True:
            rho = np.outer(np.array(Statevector(qc[i])), np.array(Statevector(qc[i])).conj())
            dep_channel = bipartite_gate(depolarizing_kraus(p), depolarizing_kraus(q))
            rho = channel(rho,dep_channel)

            U = ChangeBasis([[1, 0],[0, -1]], basis[Alice_basis[i]])
            U = np.kron(np.kron(np.eye(2), np.eye(2)), U)
            rho = np.matmul(np.matmul(U, rho), U.conj().T)
            
            U = ChangeBasis([[1, 0],[0, -1]], basis[Bob_basis[i]])
            U = np.kron(np.eye(2), np.kron(U, np.eye(2)))
            rho = np.matmul(np.matmul(U, rho), U.conj().T)

            outcome = measure_depolarization(rho)
            measured_bits.append(outcome)
            
        
        elif depolarization == False:
            U = ChangeBasis([[1, 0],[0, -1]], basis[Alice_basis[i]])
            U = Operator(U)
            qc[i].unitary(U, 0)
            U = ChangeBasis([[1, 0],[0, -1]], basis[Bob_basis[i]])
            U = Operator(U)
            qc[i].unitary(U, 1)
            
            qc[i].measure(0,0)
            qc[i].measure(1, 1)
            qc[i].measure(2, 2)
            outcome = measure(qc[i])
            measured_bits.append(outcome)


        

    return Alice_basis, Bob_basis, measured_bits
        

In [15]:
def extract_matching_bits(Alice_basis, Bob_basis, measured_bits):
    weak_key_alice = []
    weak_key_bob = []
    
    for i in range(len(Alice_basis)):
        if Alice_basis[i] == Bob_basis[i]:
            weak_key_alice.append(measured_bits[i][2])
            weak_key_bob.append(measured_bits[i][1])
    
    return weak_key_alice, weak_key_bob

def calculate_difference_percentage(weak_key_alice, weak_key_bob):

    weak_key_alice = [int(bit) for bit in weak_key_alice]
    weak_key_bob = [int(bit) for bit in weak_key_bob]

    differing_bits = sum(1 for i in range(len(weak_key_alice)) if weak_key_alice[i] != weak_key_bob[i])

    difference_percentage = (differing_bits / len(weak_key_alice)) * 100

    return difference_percentage

def compare_and_display_key_difference(weak_key_alice, weak_key_bob):

    difference_percentage = calculate_difference_percentage(weak_key_alice, weak_key_bob)
    print(f"Alice's key: {weak_key_alice}")
    print(f"Bob's key: {weak_key_bob}")
    print(f"Difference between Alice's and Bob's weak keys: {difference_percentage:.2f}%")
    print()

In [16]:
def E91(sequence_leangh, shellBlock, shellIteration, eve = False, depolarization = False, p = 1/2, q = 1/2):
    
    qc = [GenerateBellState(eve) for i in range(sequence_leangh)]
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    theta = 0
    basis = []
    
    basis.append(np.cos(theta) * X + np.sin(theta) * Z)
    basis.append(np.cos(theta + np.pi/4) * X + np.sin(theta + np.pi/4) * Z)
    basis.append(np.cos(theta + np.pi/2) * X + np.sin(theta + np.pi/2) * Z)
    basis.append(np.cos(theta + 3*np.pi/4) * X + np.sin(theta + 3*np.pi/4) * Z)

    Alice_basis, Bob_basis, measured_bits = calculate(sequence_leangh, qc, basis, depolarization, p, q)

    S = BellTest(sequence_leangh, Alice_basis, Bob_basis, measured_bits)
    print("Bell test's experimental value:", S)
    print()
    
    #----------------------------------------------------------#
    
    weak_key_alice, weak_key_bob = extract_matching_bits(Alice_basis, Bob_basis, measured_bits)
    weak_key_alice = ''.join(weak_key_alice)
    weak_key_bob = ''.join(weak_key_bob)
    print("weak keys:")
    compare_and_display_key_difference(weak_key_alice, weak_key_bob)

    #----------------------------------------------------------#
    
    string1 = list(weak_key_alice)
    string2 = list(weak_key_bob)
    s1, s2 = shell_protocol(string1, string2, shellBlock, shellIteration)
    reconcilated_key_Alice = ''.join(s1)
    reconcilated_key_Bob = ''.join(s2) 
    print("reconcilated keys:")
    compare_and_display_key_difference(reconcilated_key_Alice, reconcilated_key_Bob)

    #----------------------------------------------------------#
    
    final_key_length = 32  
    family_size = 256
    key_length = len(weak_key_alice)
    hash_family = generate_hash_family(final_key_length, key_length, family_size)
    
    chosen_hash_index = random.randint(0, family_size - 1)
    print(f"Alice's chosen hash index: {chosen_hash_index}")

    H_alice = hash_family[chosen_hash_index]
    H_bob = hash_family[chosen_hash_index]

    hashed_key_alice = universal_hash_with_index(reconcilated_key_Alice, H_alice)
    hashed_key_bob = universal_hash_with_index(reconcilated_key_Bob, H_bob)
    #print(f"Hashed key for Alice: {hashed_key_alice}")
    #print(f"Hashed key for Bob: {hashed_key_bob}")

    secret_key = get_random_bytes(16)
    final_secure_key_alice = secure_hash_using_hmac(secret_key, hashed_key_alice)
    final_secure_key_bob = secure_hash_using_hmac(secret_key, hashed_key_bob)
    print(f"Final secure key for Alice: {final_secure_key_alice}")
    print(f"Final secure key for Bob: {final_secure_key_bob}")


    

E91 is the final function and by calling this simple function you can simulate the ***E91*** protocol.

E91(sequence_leangh, shellBlock, shellIteration, eve = False, depolarization = False, p = 1/2, q = 1/2):

lets explain what each intety means:

sequence_leangh sequence_leangh is leangh of the EPR pairs that, EPR source will send to both paries

shellBlock is a parameter in shell protocol wich determines each asking session betwean Alice and Bob contains a block with this size

shellIteration: the number of times shell protocol is applied to the string.

eve: determines presens of eve

depolarization: determines presens of depolarization nose

p: parameter that determines with what probabilty the first qubit is depolarized
q: parameter that determines with waht probabilty the second qubit is depolarized 

bellow you can see some examples of using this function and their results.



In [21]:
E91(sequence_leangh, shellBlock, shellIteration)

Bell test's experimental value: 2.771123685610079

weak keys:
Alice's key: 0010110010111101001011111100001000001000100101110111100111000000000010111010010010001101001001100010000000101110101001011100001010101110011011110000010110101100101000111110011011000100110011000011000010010010111101111011110101000011010100111111110000101010101100111111100100000010000101000101101100110111011010110101010111100101010100110000101011110010001111101000011100100010101010100110111110110011100101001110110110101101110010111110000110010001110010010001100010011010101000010101001110000000010100010010111011010000000010000011000101100000011000010100010100000010011111000011011110001001110011100011110011010010001111000101000010000111010010010000111111111000111010111001100001101000010101001001011100011110100011000110001110111100010010001010001010110001001101110010011111000001000000110101001001000011000000101100001000000000011000101100101001101000111001111000101111110000011011110110111
Bob's key: 001011001011110100

In [22]:
E91(sequence_leangh, shellBlock, shellIteration, depolarization = True, p = 0.1, q = 0.1)

Bell test's experimental value: 2.343319760302724

weak keys:
Alice's key: 11111111000100100001010011100101010100110100111000100100100101111000100110001000111010111100010111100011010001111000100000101001011001111101000000001011011010100111101011110011101010101000000010000011001111110010000100011101110111000111101010001111011010110101110101011100000010000110000011010101111001110000110110110111001011111101111100000101000000111010011010001001010110000001100111010010111010011000001101111101000111000001000010100111111111101100011111000011010000000111100110100011011001101001010101001101011101101000010001011001010101111010100111101111001000111101110010101101010010111001110111101001010111101000100110011000100111000101101011110101111010001101010101110000101101000011100010110110011100101001011000111010101101101111100011010111000000111011111011000101011111010001101111111010111100000010110000110000011100011000011100000001111100111
Bob's key: 11011111000100100011010011100101010100010100111000100100

In [23]:
E91(sequence_leangh, shellBlock, shellIteration, eve = True)

Bell test's experimental value: 1.4558084624301495

weak keys:
Alice's key: 101110001010011001110111101110011111011111110011111010100101010100110001110011100111001011111100101111001000000000101011010000101101111010110101111110010100111110010001011100001101111001011010111010010000010001000010010111011011011111110001000100110110110000111110011101110101101010011001001001000000001101010101000100001010010110101010000111110011110100001101110001001111001000010010010111101001101110111101111110101110101111101110110101111101100001110111010100001100010110010100010000011110111011110111100101111100011101011000100011111001000101010101101100011100010111010010100101110000000001100000100011001011101101000001000111010000110111011001000110101010010100000110001110101101010100000101100001110001001010111011101011110000111001010010011101000111101100001110000100100000111001110100010010000100011010111110001001000101011010111111001101100110010111010000011111011001110101
Bob's key: 001110001010011000110111101110

In [24]:
E91(sequence_leangh, shellBlock, shellIteration, eve = True, depolarization = True, p = 0.1, q = 0.1)

Bell test's experimental value: 1.074896013055035

weak keys:
Alice's key: 000110010011101101111001101000010001011100110000011101111001011011000110010000000001100011011010010100100101001111001000110011011110100110011010011011000110000010011011111110101110110111010010011101101011100001000110101110011011000000110010101100101111001101000000000110101011011101011011111011000111011000011110011010111110000001010111001011101110010110101010111010100110111010011000100000110100001111010101001001011101010011011010000000101101101001100100001011111110011011101011111010111101110101111011100000110111101011110010010101010001100010100000011110001010000110100100000011000000111000111101111010000111010100010110000001110010001110110100000000110110001110111101000001100000100111101101110011100000000011011000101110110000011011011110010111101101000011010000100110001010101010100010000010111101010100101101010000100101000101000001000111100001110010011110111010
Bob's key: 0001100000111011001010011010000100010011001